In [1]:
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm 
from bs4 import BeautifulSoup 
from newspaper import Article 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# scap

In [5]:
def scrap():
    page = 0
    scraped_data  = []
    
    while True:
        page += 1

        main_page_url = f"https://www.shana.ir/archive?pi={page}&ms=0" 

        html = requests.get(main_page_url).text
    
        soup = BeautifulSoup(html , features = 'lxml' )

        links = soup.find_all('h3')
        
        if len(links) == 0:
            break
        
        for link in tqdm(links) :
            page_url = 'http://www.shana.ir/'+ link.a['href']
            try :
                
                article = Article(page_url)
                article.download()
                article.parse()
                scraped_data.append( {'url': page_url ,'text':article.text , 'title':article.title} )
                    
            except :
                print (f"failed process page : {page_url}")    
            
    df = pd.DataFrame(scraped_data)
    df.to_csv('shana.csv')    
            

# scrap()


# tfidf

In [6]:

corpus = pd.read_csv('shana.csv', index_col= 0)

In [7]:
# count of docs

print(f'cout of corpus data: {len(corpus)}')

cout of corpus data: 987


In [8]:
corpus.shape

(987, 3)

In [9]:
corpus.columns

Index(['url', 'text', 'title'], dtype='object')

In [10]:
corpus.head(5)

,url,text,title
0,http://www.shana.ir//news/458247/%D8%A7%DB%8C%...,به گزارش شانا به نقل از شرکت ملی مناطق نفت‌خیز...,ایجاد سازمان رسمی برای تلمبه‌خانه نفت گوره C گ...
1,http://www.shana.ir//news/458246/%D8%AA%D9%88%...,به گزارش شانا به نقل از شرکت بهره‌برداری نفت و...,تولید ایمن و پایدار پیش درآمد فعالیت در صنایع ...
2,http://www.shana.ir//news/458245/%DA%A9%D8%A7%...,به گزارش شانا به نقل از شرکت مناطق نفت‌خیز جنو...,کارکنان عملیاتی نباید دغدغه‌ای در امور درمان د...
3,http://www.shana.ir//news/458244/%D8%A7%D9%86%...,به گزارش شانا به نقل از شرکت ملی گاز ایران، مت...,انتصاب سرپرست شرکت گاز استان کرمانشاه
4,http://www.shana.ir//news/458243/%DA%A9%D8%B3%...,به گزارش شانا به نقل از شرکت گاز استان خراسان ...,کسب رتبه نخست نشان تعالی اچ‌اس‌ئی از سوی شرکت ...


In [11]:
# tf-idf corpus

# skip docs without text
docs = corpus.loc[corpus.text.isnull() == False]['text']

vectorizer = TfidfVectorizer()
tfidf_docs = vectorizer.fit_transform(docs)

In [12]:
# tf-idf query

query = 'نفت'

tfidf_query = vectorizer.transform([query])[0]

# similarities

cosines = []

for d in tqdm(tfidf_docs):
  cosines.append(float(cosine_similarity(d, tfidf_query)))
  
# sorting

k = 10
sorted_ids = np.argsort(cosines)

for i in range(k):
  cur_id = sorted_ids[-i-1]
  cur_doc = corpus.iloc[cur_id]
  print(cosines[cur_id], cur_doc['title'])

956it [00:01, 706.90it/s] 


0.32687950058103177 تأکید هلدینگ خلیج فارس بر تکمیل طرح‌ها پیش از موعد مقرر
0.32501811666855246 بازدید مدیرعامل شرکت ملی گاز ایران از بیست‌وششمین نمایشگاه بین‌المللی نفت
0.29721872028460083 قیمت بنزین افزایش نمی‌یابد
0.2853719699379025 تأمین ۸۵ درصدی تجهیزات ایستگاه‌های تقویت فشار گاز از سوی سازندگان داخلی
0.2801802871105066 جهش ۱۸ درصدی صادرات نفت خام عمان
0.27654704385870194 هلدینگ خلیج فارس در توسعه میدان‌ها از توان داخلی استفاده می‌کند
0.2753997238821162 ایران ظرفیت‌های لازم را برای تأمین نیازهای نفتی و گازی پاکستان دارد
0.25681400287810946 زمینه‌سازی شرکت مهندسی و توسعه گاز برای حضور در عرصه بین‌المللی
0.25264357485028965 دیدار مدیرعامل شرکت ملی نفت ایران با معاون نخست‌وزیر قزاقستان
0.25230421031920897 درآمد ۱۳.۵ میلیارد دلاری هلدینگ خلیج فارس در ۱۴۰۱
